In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
from typing import Generator, List
import pickle

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English

In [3]:
with open(r"../data/interim/review_classes.pkl", "rb") as input_file:
    review_classes = pickle.load(input_file)

In [4]:
review_classes["NEG"]

['Good location on the NW of town, especially if youre going to Antiguena spanish school. 100q for good sized private twin room with shared bathroom - great price Nice ish garden with a swing and hamock Basic bathrooms - very tatty but has hot showers always (if you keep the water flow slow) Super basic, pretty dirty kitchenette Loads of cats and dogs  Its very cheap and basic. If thats what your after, its ok',
 'We went to the hotel directly to book a room. The receptionist at first gave us a really high price then lowered it. When we asked how much it costs in qetales she gave us an 8.2 comission rate to dollars and I knew she was randomly goving us a non existing rate so then she "lowered" it to the real rate. After we agreed that the price includes breakfast the following morning we get to breakfast and received two plain pieces of toast and fruit while theother tables received also eggs and plantines. So I asked if we also get eggs abd plantines and she replied "no". How come we 

In [5]:
positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']

In [6]:
negative_reviews

['Good location on the NW of town, especially if youre going to Antiguena spanish school. 100q for good sized private twin room with shared bathroom - great price Nice ish garden with a swing and hamock Basic bathrooms - very tatty but has hot showers always (if you keep the water flow slow) Super basic, pretty dirty kitchenette Loads of cats and dogs  Its very cheap and basic. If thats what your after, its ok',
 'We went to the hotel directly to book a room. The receptionist at first gave us a really high price then lowered it. When we asked how much it costs in qetales she gave us an 8.2 comission rate to dollars and I knew she was randomly goving us a non existing rate so then she "lowered" it to the real rate. After we agreed that the price includes breakfast the following morning we get to breakfast and received two plain pieces of toast and fruit while theother tables received also eggs and plantines. So I asked if we also get eggs abd plantines and she replied "no". How come we 

In [7]:
def sentences_to_words(sentences: List[str]) -> List[List[str]]:
    words = []
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        words.append(simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación
    return words

In [8]:
print(negative_reviews[1])
negative_words = sentences_to_words(negative_reviews)
positive_words = sentences_to_words(positive_reviews)
print(negative_words[1])

We went to the hotel directly to book a room. The receptionist at first gave us a really high price then lowered it. When we asked how much it costs in qetales she gave us an 8.2 comission rate to dollars and I knew she was randomly goving us a non existing rate so then she "lowered" it to the real rate. After we agreed that the price includes breakfast the following morning we get to breakfast and received two plain pieces of toast and fruit while theother tables received also eggs and plantines. So I asked if we also get eggs abd plantines and she replied "no". How come we didn't get the same breakfast when we paid for it?? Not a good experience at all at the hotel. There are much better places in Antigua to stay.
['we', 'went', 'to', 'the', 'hotel', 'directly', 'to', 'book', 'room', 'the', 'receptionist', 'at', 'first', 'gave', 'us', 'really', 'high', 'price', 'then', 'lowered', 'it', 'when', 'we', 'asked', 'how', 'much', 'it', 'costs', 'in', 'qetales', 'she', 'gave', 'us', 'an', 'c

In [9]:
def remove_stopwords(documents: List[List[str]]) -> List[List[str]]:
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords.words('english')]
            for doc in documents]

In [10]:
print(negative_words[1])
negative_words_wo_stopwords = remove_stopwords(negative_words)
print(negative_words_wo_stopwords[1])

['we', 'went', 'to', 'the', 'hotel', 'directly', 'to', 'book', 'room', 'the', 'receptionist', 'at', 'first', 'gave', 'us', 'really', 'high', 'price', 'then', 'lowered', 'it', 'when', 'we', 'asked', 'how', 'much', 'it', 'costs', 'in', 'qetales', 'she', 'gave', 'us', 'an', 'comission', 'rate', 'to', 'dollars', 'and', 'knew', 'she', 'was', 'randomly', 'goving', 'us', 'non', 'existing', 'rate', 'so', 'then', 'she', 'lowered', 'it', 'to', 'the', 'real', 'rate', 'after', 'we', 'agreed', 'that', 'the', 'price', 'includes', 'breakfast', 'the', 'following', 'morning', 'we', 'get', 'to', 'breakfast', 'and', 'received', 'two', 'plain', 'pieces', 'of', 'toast', 'and', 'fruit', 'while', 'theother', 'tables', 'received', 'also', 'eggs', 'and', 'plantines', 'so', 'asked', 'if', 'we', 'also', 'get', 'eggs', 'abd', 'plantines', 'and', 'she', 'replied', 'no', 'how', 'come', 'we', 'didn', 'get', 'the', 'same', 'breakfast', 'when', 'we', 'paid', 'for', 'it', 'not', 'good', 'experience', 'at', 'all', 'at',

In [11]:
positive_words_wo_stopwords = remove_stopwords(positive_words)

In [12]:
def learn_bigrams(documents: List[List[str]]) -> List[List[str]]:
    # We learn bigrams
    #  https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
    bigram = Phrases(documents, min_count=5, threshold=10)

    # we reduce the bigram model to its minimal functionality
    bigram_mod = Phraser(bigram)

    # we apply the bigram model to our documents
    return bigram_mod


def create_bigrams(bigram_model, documents: List[List[str]]):
    return [bigram_model[doc] for doc in documents]

In [13]:
bigram_model = learn_bigrams(negative_words_wo_stopwords + positive_words_wo_stopwords)
negative_words_wo_stopwords_bigrams = create_bigrams(bigram_model, negative_words_wo_stopwords)

In [14]:
print(negative_words_wo_stopwords[10])
print(negative_words_wo_stopwords_bigrams[10])

['stayed', 'total', 'nearly', 'weeks', 'eight', 'visits', 'next', 'visit', 'already', 'reserved', 'family', 'friends', 'strangers', 'become', 'friends', 'five', 'star', 'property', 'conversion', 'hacienda', 'hotel', 'slap', 'dash', 'haphazard', 'times', 'much', 'street', 'noise', 'wifi', 'wonky', 'back', 'recommend', 'friends', 'oasis', 'expectations', 'courtyards', 'walk', 'ways', 'hang', 'dense', 'flowering', 'vines', 'spring', 'summer', 'added', 'flowering', 'shrubs', 'trees', 'perfumes', 'small', 'world', 'way', 'chanel', 'aspires', 'fountain', 'main', 'courtyard', 'adds', 'bouquet', 'soundtrack', 'splashing', 'water', 'perfect', 'place', 'sip', 'beer', 'read', 'book', 'fall']
['stayed', 'total', 'nearly', 'weeks', 'eight', 'visits', 'next', 'visit', 'already', 'reserved', 'family_friends', 'strangers', 'become', 'friends', 'five_star', 'property', 'conversion', 'hacienda', 'hotel', 'slap', 'dash', 'haphazard', 'times', 'much', 'street_noise', 'wifi', 'wonky', 'back', 'recommend', 

In [15]:
def lemmatization(nlp: English, texts: List[List[str]], allowed_postags: List = None) -> List[List[str]]:
    if allowed_postags is None:
        allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']

    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [21]:
nlp = spacy.load('en_core_web_trf')

In [22]:
negative_words_wo_stopwords_bigrams_pos = lemmatization(nlp, negative_words_wo_stopwords_bigrams)

In [27]:
print(negative_words_wo_stopwords_bigrams[3])
print(negative_words_wo_stopwords_bigrams_pos[3])

['arrived_late', 'night', 'delayed', 'bus', 'journey', 'original', 'accommodation', 'given', 'away', 'place', 'could', 'find', 'could', 'accommodate', 'grateful', 'priced', 'private', 'room', 'wifi', 'seems', 'decent', 'value', 'however', 'rooms', 'incredibly', 'basic', 'unclean', 'mosquitos', 'everywhere', 'insects', 'floor', 'bathroom', 'key', 'door', 'apparently', 'however', 'lock', 'inside', 'although', 'feel', 'secure', 'place', 'stay', 'night_guard', 'big', 'front_door', 'locked', 'throughout', 'night', 'staff', 'speak_english', 'limited_spanish', 'tested', 'see', 'much', 'else', 'bathroom', 'bedroom', 'feel', 'comfortable', 'morning', 'came']
['night', 'delay', 'bus', 'journey', 'original', 'accommodation', 'give', 'place', 'find', 'accommodate', 'grateful', 'price', 'private', 'room', 'wifi', 'seem', 'decent', 'value', 'however', 'room', 'incredibly', 'basic', 'unclean', 'mosquito', 'everywhere', 'insect', 'floor', 'bathroom', 'key', 'door', 'apparently', 'however', 'lock', 'in

In [38]:
def tokenize(documents: List[str], bigram_model) -> List[List[str]]:

    document_words = sentences_to_words(documents)
    document_words = remove_stopwords(document_words)
    document_words = create_bigrams(bigram_model, document_words)
    document_words = lemmatization(nlp, document_words)

    return document_words

In [39]:
positive_words = tokenize(positive_reviews, bigram_model)

KeyboardInterrupt: 

In [ ]:
negative_words = tokenize(negative_reviews, bigram_model)

In [39]:
print(positive_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
with open(r"../data/interim/positive_words.pkl", "wb") as output_file:
    pickle.dump(positive_words, output_file)

with open(r"../data/interim/negative_words.pkl", "wb") as output_file:
    pickle.dump(negative_words, output_file)
    


In [40]:
with open(r"../data/interim/bigram_model.pkl", "wb") as output_file:
    pickle.dump(bigram_model, output_file)